In [ ]:
!pip install --upgrade gspread

In [ ]:
from google.colab import auth
auth.authenticate_user()
from google.auth import default
creds, _ = default()
from oauth2client.service_account import ServiceAccountCredentials

import requests
import json
import math
import pandas as pd
import numpy as np
import re

import os

import gspread
gc = gspread.authorize(creds)

In [ ]:
#reference: https://youtu.be/q0EgxM2iAII

In [ ]:
# specify the URL of the website you want to scrape
url = 'https://id.wikipedia.org/wiki/Daftar_kabupaten_dan_kota_di_Indonesia'

# read all tables from the webpage into a list of dataframes
tables = pd.read_html(url)

# initialize an empty list to hold the first columns from all tables
city_columns = []

# loop through the tables and extract the first column from each table
for i in range(1, 39):
    df = tables[i]
    city_column = df.iloc[:, 1]
    city_columns.append(city_column)

# concatenate the first columns vertically into a single series
combined_city_column = pd.concat(city_columns, ignore_index=True)

# create a new dataframe with only the first column of `df`
df_city_column = pd.DataFrame(combined_city_column)

df_city = df_city_column.reset_index(drop=True)
df_city.columns = ['org_city_helper']
df_city['org_city_helper'] = df_city['org_city_helper'].apply(lambda x: 'Kabupaten ' + x if not x.startswith(('Kabupaten', 'Kota')) else x)
df_city['org_city_helper'] = df_city['org_city_helper'].str.replace("Administrasi ","")
df_city = df_city[~df_city['org_city_helper'].str.contains('Kabupaten/kota')]
df_city['org_city_abb_helper'] = df_city['org_city_helper'].str.replace("Kabupaten |Kota |Administrasi ","")
df_city = df_city.reset_index(drop=True)
df_city

<ipython-input-221-43c60398c3c4>:27: FutureWarning: The default value of regex will change from True to False in a future version.
  df_city['org_city_abb_helper'] = df_city['org_city_helper'].str.replace("Kabupaten |Kota |Administrasi ","")


,org_city_helper,org_city_abb_helper
0,Kabupaten Aceh Barat,Aceh Barat
1,Kabupaten Aceh Barat Daya,Aceh Barat Daya
2,Kabupaten Aceh Besar,Aceh Besar
3,Kabupaten Aceh Jaya,Aceh Jaya
4,Kabupaten Aceh Selatan,Aceh Selatan
...,...,...
509,Kabupaten Raja Ampat,Raja Ampat
510,Kabupaten Sorong,Sorong
511,Kabupaten Sorong Selatan,Sorong Selatan
512,Kabupaten Tambrauw,Tambrauw


In [ ]:
# specify the URL of the website you want to scrape
url = 'https://id.wikipedia.org/wiki/Daftar_kabupaten_dan_kota_di_Indonesia'

# read all tables from the webpage into a list of dataframes
tables = pd.read_html(url)

# initialize an empty list to hold the first columns from all tables
number_columns = []

# loop through the tables and extract the first column from each table
for i in range(0, 1):
    df = tables[i]
    number_column = df.iloc[:, 5]
    number_columns.append(number_column)

# concatenate the first columns vertically into a single series
combined_number_column = pd.concat(number_columns, ignore_index=True)

# create a new dataframe with only the first column of `df`
df_column = pd.DataFrame(combined_number_column)
df_column = df_column[:-1]

# remove the index column
df_column = df_column.reset_index(drop=True)

df_column_list = df_column.values.flatten().tolist()
df_column_list

In [ ]:
# specify the URL of the website you want to scrape
url = 'https://id.wikipedia.org/wiki/Daftar_kabupaten_dan_kota_di_Indonesia'

# read all tables from the webpage into a list of dataframes
tables = pd.read_html(url)

# initialize an empty list to hold the first columns from all tables
province_columns = []

# loop through the tables and extract the first column from each table
for i in range(0, 1):
    df = tables[i]
    province_column = df.iloc[:, 2]
    province_columns.append(province_column)

# concatenate the first columns vertically into a single series
combined_province_column = pd.concat(province_columns, ignore_index=True)

# create a new dataframe with only the first column of `df`
df_province_column = pd.DataFrame(combined_province_column)

df_province_column = df_province_column[~df_province_column['Provinsi'].str.contains('Jumlah')]

# remove the index column
df_province_column = df_province_column.reset_index(drop=True)

# display the first column as a pandas dataframe
df_province_column

copied_rows = []  # Initialize an empty list to hold the copied rows

for i, row in df_province_column.iterrows():
    num_copies = df_column_list[i]
    copies = [row] * num_copies
    copied_rows.extend(copies)

# Create a new DataFrame with the copied rows
df_copied_rows = pd.DataFrame(copied_rows)

# Reset the index of the new DataFrame
df_province = df_copied_rows.reset_index(drop=True)

df_province.columns = ['org_province_helper']
df_province['org_province_helper'] = df_province['org_province_helper'].str.replace("Sumatra","Sumatera")
df_province

,org_province_helper
0,Aceh
1,Aceh
2,Aceh
3,Aceh
4,Aceh
...,...
509,Papua Barat Daya
510,Papua Barat Daya
511,Papua Barat Daya
512,Papua Barat Daya


In [ ]:
result = pd.concat([df_city, df_province], axis=1)
result

,org_city_helper,org_city_abb_helper,org_province_helper
0,Kabupaten Aceh Barat,Aceh Barat,Aceh
1,Kabupaten Aceh Barat Daya,Aceh Barat Daya,Aceh
2,Kabupaten Aceh Besar,Aceh Besar,Aceh
3,Kabupaten Aceh Jaya,Aceh Jaya,Aceh
4,Kabupaten Aceh Selatan,Aceh Selatan,Aceh
...,...,...,...
509,Kabupaten Raja Ampat,Raja Ampat,Papua Barat Daya
510,Kabupaten Sorong,Sorong,Papua Barat Daya
511,Kabupaten Sorong Selatan,Sorong Selatan,Papua Barat Daya
512,Kabupaten Tambrauw,Tambrauw,Papua Barat Daya


In [ ]:
result_list = result.values.tolist()

wb = gc.open_by_url("...")
wscp = wb.worksheet("org_city-province_helper")
wscp.clear()
wscp.update("A2", result_list)